In [36]:
import pandas as pd
import numpy as np
import sklearn
import json
import time
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import eli5
import shap
from eli5.sklearn import PermutationImportance

In [37]:
from search_query import search
from report_get import report
from job_id_extract import job_id_extract
from search_write_to_file import search_and_write
from dict_make import dict_make
from report_get_write_to_file import report_write
from feature_extract import feature_extraction
from target_extract import threat_score_extract
from make_df import frame_maker

In [38]:
from the_functions import *

In [5]:
dates_from = ['2020-09-28 00:00', '2020-09-29 00:00', '2020-9-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00']
dates_to = ['2020-09-29 00:00', '2020-09-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00', '2020-10-21 00:00']

# Introduction, Business Understanding

Questions:
- How does Hybrid Analysis construct its threat score for URL's?
    - Can a generalized model be made based on the reports their Falcon Sandbox platform produces?
- What features are important to Hybrid Analysis URL threat score?
    - Can these features be determined, and then in turn, easily be extracted with guidance from their reports.
    - If reverse engineering isn't possible, can generealized insights still be produced?
    
Objectives:
- Creae a model that interprets Hybrid Analysis reports to determine threat score on URL's
- Create a feature list of Hybrid Analysis most informative features
- If the above are successful, deploy a reverse engineered model

Success Criteria:
- Either a functional model or insights into Hybrid Analysis feature importance for URL's that could guide another project

Hybrid Analysis:
- a website that allows users to submit malware, url's executables, html, etc for a proprietary "threat score" analysis.

# What have I done so far?

made search query and writing to file functions
made job_id extracting function that takes a dictionary input

can I get the things I need?
I need
- large set of malicious URL's
    - I can get these using my search and write function using a large enough window
- the malicious URLS's accompanying JOB id's
    - I can get these by extraccting the job ID's using my function on the dictionary
- Now I have job id's, target values, now I need features.
    - I need to use report get to get reports for each of my malicios URLS's I am storing that have the features
- Researching what is exactly contained in the feature reports
    - MUTEX (Mutual Exclusion) Locking down parts of code by keeping it in use, sometimes bloking visibility of other programs and making it immutable to outside other programs, think "file is in use somewhere else, cannot delete"

In [10]:
# dates_from = ['2020-09-28 00:00', '2020-09-29 00:00', '2020-9-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00','2020-10-21 00:00', '2020-10-22 00:00', '2020-10-23 00:00', '2020-10-24 00:00', '2020-10-25 00:00', '2020-10-26 00:00', '2020-10-27 00:00', '2020-10-28 00:00','2020-10-29 00:00', '2020-11-01 00:00', '2020-11-02 00:00', '2020-11-03 00:00', '2020-11-04 00:00', '2020-11-05 00:00', '2020-11-06 00:00', '2020-11-07 00:00', '2020-11-08 00:00', '2020-11-09 00:00', '2020-11-10 00:00', '2020-11-11 00:00', '2020-11-12 00:00', '2020-11-13 00:00', '2020-11-14 00:00', '2020-11-15 00:00', '2020-11-16 00:00', '2020-11-17 00:00', '2020-11-18 00:00', '2020-11-19 00:00', '2020-11-20 00:00', '2020-11-21 00:00', '2020-11-22 00:00', '2020-11-23 00:00', '2020-11-24 00:00', '2020-11-25 00:00']
# dates_to = ['2020-09-29 00:00', '2020-09-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00', '2020-10-21 00:00', '2020-10-22 00:00', '2020-10-23 00:00', '2020-10-24 00:00', '2020-10-25 00:00', '2020-10-26 00:00', '2020-10-27 00:00', '2020-10-28 00:00','2020-10-29 00:00', '2020-11-01 00:00', '2020-11-02 00:00', '2020-11-03 00:00', '2020-11-04 00:00', '2020-11-05 00:00', '2020-11-06 00:00', '2020-11-07 00:00', '2020-11-08 00:00', '2020-11-09 00:00', '2020-11-10 00:00', '2020-11-11 00:00', '2020-11-12 00:00', '2020-11-13 00:00', '2020-11-14 00:00', '2020-11-15 00:00', '2020-11-16 00:00', '2020-11-17 00:00', '2020-11-18 00:00', '2020-11-19 00:00', '2020-11-20 00:00', '2020-11-21 00:00', '2020-11-22 00:00', '2020-11-23 00:00', '2020-11-24 00:00', '2020-11-25 00:00', '2020-11-26 00:00']

In [11]:
# len(search('2020-09-28 00:00', '2020-09-29 00:00', 5))

In [12]:
# search_and_write(dates_from, dates_to)

In [13]:
# with open('data/data0.txt') as json_file:
#     data = json.load(json_file)
# data

In [6]:
my_dict = dict_make(dates_from)

In [7]:
my_list = job_id_extract(my_dict)

In [16]:
# with open('reports/reports5f7e47fdb8e47e68de0a152a.txt') as json_file:
#     data = json.load(json_file)
# data

In [17]:
# rep = report("5f9a05839fb71b523c1a2b59")

In [18]:
# rep

In [19]:
# #count unique categories
# category_list = []
# for i in rep['response']['Event']['Attribute']:
#     category_list.append(i['category'])
# set(category_list)
# # one_hot = []
# # if 'Artifacts dropped' in set(list):
# #     one_hot.append(1)
    
# # else:
# #     one_hot.append(0)
# # one_hot

# # for i in set(list):
# #     print('this could be it')

In [20]:
# #count unique types
# clist = []
# for i in rep['response']['Event']['Attribute']:
#     clist.append(i['type'])
# clist
# set(clist)

In [21]:
# list_type = []
# list_category = []
# for i in my_list:
#     with open(f'reports/reports{i}.txt') as json_file:
#         features = []
#         job_id = i
#         x = json.load(json_file)
        
#         for i in x['response']['Event']['Attribute']:
#             list_category.append(i['category'])
        
        
#         for i in x['response']['Event']['Attribute']:
#             list_type.append(i['type'])
# print(set(list_type), set(list_category))

In [22]:
# p = set(clist) - set(list_type)
# p

In [23]:
# for i in os.listdir('reports'):
#     try:
#         with open(f'{i}') as json_file:
#             x = json.load(json_file)
#             print(x)
#     except:
#         print(i)

In [24]:
# # can open all files in reports - this will become my core feature extraction
# a_dict = {}
# category_list = ['Artifacts dropped', 'External analysis', 'Network activity', 'Payload delivery', 'Payload installation', 'Persistence mechanism']
# types_list = ['filename|md5', 'user-agent', 'domain|ip', 'mutex', 'ip-dst', 'regkey|value', 'comment', 'filename|sha512', 'domain', 'filename|sha256', 'link', 'filename|sha1']
# for i in my_list:
#     with open(f'reports/reports{i}.txt') as json_file:
#         features = []
#         job_id = i
#         x = json.load(json_file)
        
        
#         #acg length of attribute vlaues
#         lenRep = len(x['response']['Event']['Attribute'])
#         list_of_lengths = []
#         sum = 0
        
#         for i in x['response']['Event']['Attribute']:
#             list_of_lengths.append(i['value'])
#         for i in list_of_lengths:
#             sum += len(i)
#         avg_len = (sum/lenRep)
        
#         features.append(avg_len)
        
        
#         # total length of attribute values
#         list_of_lengths = []
#         sum = 0
        
#         for i in x['response']['Event']['Attribute']:
#             list_of_lengths.append(i['value'])
#         for i in list_of_lengths:
#             sum += len(i)
        
#         features.append(sum)
        
#         # count of categories
#         list = []
#         for i in x['response']['Event']['Attribute']:
#             list.append(i['category'])
#         len_list = len(list)
        
#         features.append(len_list)
        
#         # one hotting categories
#         cats = []
#         for i in x['response']['Event']['Attribute']:
#             cats.append(i['category'])
#         cats = set(cats)
#         for i in category_list:
#             if i in cats:
#                 features.append(1)
#             else:
#                 features.append(0)
        
#         types = []
#         for i in x['response']['Event']['Attribute']:
#             types.append(i['type'])
#         types = set(types)
#         for i in types_list:
#             if i in types:
#                 features.append(1)
#             else:
#                 features.append(0)
        
        
#         a_dict[f'{job_id}'] = features
# a_dict

In [8]:
a_dict = feature_extraction(my_list)

In [9]:
df = frame_maker(a_dict, my_dict)

In [10]:
df

,job_id,avg_val_len,total_val_len,category_count,Artifacts dropped,External analysis,Network activity,Payload delivery,Payload installation,Persistence mechanism,...,mutex,ip-dst,regkey|value,comment,filename|sha512,domain,filename|sha256,link,filename|sha1,target
0,5f72774a50643e7bc7460603,102.382143,86001.0,840.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,100
1,5f7274e3e6faf447dd379eca,128.876732,176690.0,1371.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,90
2,5f72734166e01f097b2eeb44,123.602110,164020.0,1327.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,55
3,5f7271bd57dba718d1561324,117.141732,59508.0,508.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,41
4,5f751bc17735bd73cb45719f,152.422642,40392.0,265.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,5f8f786fab21e820cb349369,150.880597,70763.0,469.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,63
155,5f8f786f2b6ec67ce2322643,166.184154,77608.0,467.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,63
156,5f8f786f1e6bbf117f3f48f9,153.259341,69733.0,455.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,63
157,5f8f7137dcd73c04135d12df,121.845802,79809.0,655.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,81


# features

need to save a job ID for any kind of feature extraction, probably create a dictionary? creating values for columns? 'count_unique_categories':5, 'total_categories':153

couunt unique categories: each report has categories of nefarious things a url possesses. reachable by rep['response']['Event']['Attribute'] -> i['category']

```
list = []
for i in rep['response']['Event']['Attribute']:
    list.append(i['category'])
list
set(list)
```

avg length of categories values: these are usually just a hash, but it's still information, found in rep['response']['Event']['Attribute']['value']
```
lenRep = len(rep['response']['Event']['Attribute'])
list = []
sum = 0
for i in rep['response']['Event']['Attribute']:
    list.append(i['value'])
for i in list:
    sum += len(i)
sum/lenRep
```

total length of category values : same as above but just sum
```
lenRep = len(rep['response']['Event']['Attribute'])
list = []
sum = 0
for i in rep['response']['Event']['Attribute']:
    list.append(i['value'])
for i in list:
    sum += len(i)
sum/lenRep
```

#### MUTEX IS A TYPE!!!

rep['response']['Event']['Attribute'][i] - the i here is arbitrary, but what this does is drills down to the features, the 'attributes' are the features, mutex, payload delivery, persistence mechanism etc. can just count the number of attributes. If you add ['value'] to the end of the statement it gets the values of i which is typically a something with a hash?

avg length of value in payload_delivery

- can just use both on any number of feature possibilities

max length of something/anything

distance between feature_id 1 and 2 != 2 and 3

# Functions

dict_make - makes a combined dictionary of all downloaded data files from using search_and_write

job_id_extract - loops through the dictionary created by dict_make to append the job_id to a list

report_write - takes the job_id list extracted from job_id_extract and queries hybrid_analysis for the report that contains features
report_get - the GET request used in report_write


search_write_to_file - passes the date lists to search_query and gets the returns that contain the threat scores, then saves them as dataX
search_query - the post request used in search_write_to_file

In [11]:
df = df.drop(columns = 'job_id')

In [12]:
X = df.drop(columns = 'target')
y = df['target']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=5)

In [14]:
rfr = RandomForestRegressor(max_depth=6, random_state=5)

In [15]:
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, random_state=5)

In [16]:
pred = rfr.predict(X_test)

In [17]:
error = y_test - pred
sum_error = 0
for i in error:
    sum_error += i
print(sum_error, '\n',error)

-122.57677627141302 
 26     5.833435
87    -9.465071
71     0.048401
66     6.904859
93    -6.380505
         ...   
101    3.737162
122    0.048401
124    0.903614
155    0.372492
49     0.143005
Name: target, Length: 64, dtype: float64


In [18]:
perm_model = PermutationImportance(rfr, random_state = 1).fit(X_train, y_train)
eli5.show_weights(perm_model, feature_names = X_train.columns.to_list())

Weight,Feature
0.7713 ± 0.2648,category_count
0.3163 ± 0.0317,total_val_len
0.2450 ± 0.0504,avg_val_len
0.0661 ± 0.0350,domain
0.0256 ± 0.0087,user-agent
0.0025 ± 0.0013,regkey|value
0.0019 ± 0.0006,mutex
0.0014 ± 0.0001,Payload installation
0.0014 ± 0.0000,Artifacts dropped
0.0013 ± 0.0000,Persistence mechanism
